## (GSE48213): Identifying gene expression patterns associated with different breast cancer subtypes

The dataset includes both treated (estrogen) and control conditions. 
We use Adaptive CCA to identify differences in gene expression patterns between these conditions over time.

The time points (1, 2, 4, 8, 12 hours) are not equally spaced, which is common in biological experiments. 
- Adaptive CCA should handle such non-linear time progressions 

In [32]:
import os
import pandas as pd
import numpy as np

file_path = os.path.join(os.getcwd(), "..", "data", "GSE48213")

In [ ]:
os.getcwd()

Dataset overview:

- 56 breast cancer cell lines were profiled
- The data represents gene expression levels in these cell lines
- Each cell line is in an unperturbed, baseline state


In current file: 

1. Column 1 (EnsEMBL_Gene_ID): unique identifier for each gene from the Ensembl database
2. Column 2 (e.g., MDAMB453): expression value for each gene in the specific cell line.

These are normalized read counts or FPKM/TPM values (Fragments/Transcripts Per Kilobase Million).
Higher values indicate higher expression of the gene in that cell line, zero values indicate that the gene is not expressed (or expression is below detection threshold)


In [ ]:
from utils.utils import load_data
load_data(file_path, os.getcwd())

In [35]:
output_file = os.path.join(os.getcwd(), "combined_data.txt")
data = pd.read_csv(output_file, sep="\t")

In [ ]:
data.head()

In [ ]:
data.columns


#### Preprocessing: Log2 Transformation
Gene expression values vary too much across genes and cell lines, with some genes having very high expression values and others having very low ones (sometimes even zero). This creates a skewed distribution. A log2 transformation helps to normalize this range and make the data more comparable across genes and cell lines.
- without log transformation, highly expressed genes dominate the analysis, hiding patterns in the data for moderately or lowly expressed genes.

In [38]:
# ---------- PREPROCESSING ----------  #
from preprocessing import log2_transform, classify

log2_data = log2_transform(data)
# classify(log2_data)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_distribution(log2_transformed_data: pd.DataFrame):
    """
    Plots the distribution of log2 transformed gene expression data.
    
    Arguments:
    - log2_transformed_data: DataFrame with the log2 transformed gene expression values.
    """
    # Exclude the gene ID column
    expression_data = log2_transformed_data.iloc[:, 1:]
    
    # Histogram
    plt.figure(figsize=(10, 6))
    sns.histplot(expression_data.values.flatten(), bins=50, kde=True, color='lightpink')
    plt.title('Distribution of Log2 Transformed Gene Expression Values', fontweight='bold')
    plt.xlabel('Log2 Expression Values')
    plt.ylabel('Frequency')
    plt.show()

    # Boxplot
    cmap = plt.cm.Blues
    colors = cmap(np.linspace(0.3, 1, 56))  # Generating shades of blue
    
    plt.figure(figsize=(15, 8))
    ax = sns.boxplot(data=data, palette=colors)
    
    plt.title("Boxplot of Breast Cancer Cell Line Gene Expression", fontsize=14)
    plt.xticks(rotation=90)
    plt.xlabel("Cell Line")
    plt.ylabel("Gene Expression")
    plt.show()

plot_distribution(log2_data)


In [ ]:
expression_data = pd.read_csv('cell_line_subtypes.csv', index_col=0)  # Genes x Cell lines
# select rows for which column subtype is not "Unclsasified"
subtypes = expression_data['Subtype']
subtypes = subtypes[subtypes != 'Unclassified']
expression_data = expression_data.loc[subtypes.index]
print(expression_data)


Of all data employed, no enough cell lines can be classified, leading to poor data information.

In [41]:
import gseapy as gp
import mygene

def classify_genes_in_pathways():
    """ 
    Find which genes are in which KEGG pathways.
    """
    data = pd.read_csv('combined_data.txt', sep='\t', index_col=0)
    genes = data.index.tolist()

    # Convert Ensembl IDs to gene symbols
    mg = mygene.MyGeneInfo()
    gene_info = mg.querymany(genes, scopes='ensembl.gene', fields='symbol', species='human')
    ensembl_to_symbol = {gene['query']: gene.get('symbol', '') for gene in gene_info if 'symbol' in gene} # create a dictionary of Ensembl IDs to gene symbols

    # Load KEGG pathway gene sets
    kegg_gene_sets = gp.get_library('KEGG_2021_Human')
    gene_pathway_map = {}

    for pathway, pathway_genes in kegg_gene_sets.items():
        # Find genes that are both in data and in the pathway
        common_genes = set(ensembl_to_symbol.values()).intersection(pathway_genes)
        
        # If there are common genes, add them to the gene_pathway_map
        for gene_symbol in common_genes:
            ensembl_ids = [ensembl for ensembl, symbol in ensembl_to_symbol.items() if symbol == gene_symbol]
            for ensembl_id in ensembl_ids:
                if ensembl_id not in gene_pathway_map:
                    gene_pathway_map[ensembl_id] = []
                gene_pathway_map[ensembl_id].append(pathway)

    # Create a DataFrame from the gene_pathway_map
    result_df = pd.DataFrame.from_dict(gene_pathway_map, orient='index')
    result_df.columns = [f'Pathway_{i+1}' for i in range(result_df.shape[1])]

    result_df.to_csv('gene_pathway_mappings.csv')
    
    total_genes = len(genes)
    mapped_genes = len(gene_pathway_map)
    print(f"Total genes in your data: {total_genes}")
    print(f"Genes mapped to KEGG pathways: {mapped_genes}")
    print(f"Percentage of genes mapped: {mapped_genes/total_genes*100:.2f}%")

    # Print the first few rows of the result
    print("\nFirst few gene-pathway mappings:")
    print(result_df.head())


In [ ]:
classify_genes_in_pathways()

In [43]:
gene_expression = pd.read_csv('combined_data.txt', sep='\t', index_col=0)
gene_pathway_mappings = pd.read_csv('gene_pathway_mappings.csv', index_col=0)

In [ ]:
from preprocessing import preprocessing
from sklearn.preprocessing import StandardScaler

X, Y = preprocessing(gene_expression, gene_pathway_mappings)

# Ensure X and Y are centered and scaled
scaler = StandardScaler()
X = scaler.fit_transform(X)
Y = scaler.fit_transform(Y)

# Set dimensions
n, p1 = X.shape
_, p2 = Y.shape
k = min(p1, p2, 10)  # number of canonical dimensions, adjust as needed

In [ ]:
# check if X or Y have Nan values
print(np.isnan(X).any())
print(np.isnan(Y).any())

In [66]:
from scipy.linalg import cholesky, polar

def cca_objective(X: np.ndarray, Y: np.ndarray, A: np.ndarray, B: np.ndarray) -> float:
    """ 
    Compute the objective function for CCA.

    Parameters:
    ----------
    X : np.ndarray
        The first dataset (genes with expressions).
    Y : np.ndarray
        The second dataset (pathways of the genes).
    A : np.ndarray
        The first projection matrix.
    B : np.ndarray
        The second projection matrix.
    """
    XA = X @ A
    YB = Y @ B
    corr = np.sum(XA * YB) / (n - 1)
    return -corr  # negative because we're maximizing

def cca_gradient(X: np.ndarray, Y: np.ndarray, A: np.ndarray, B: np.ndarray) -> tuple:
    """ 
    Compute the gradient of the objective function for CCA.

    Parameters:
    ----------
    X : np.ndarray
        The first dataset (genes with expressions).
    Y : np.ndarray  
        The second dataset (pathways of the genes).
    A : np.ndarray
        The first projection matrix.
    B : np.ndarray
        The second projection matrix.
    """
    assert X.shape[1] == A.shape[0] == B.shape[0]
    assert Y.shape[1] == A.shape[1] == B.shape[1]
    assert np.linalg.matrix_rank(A) == A.shape[1]
    assert np.linalg.matrix_rank(B) == B.shape[1]
    XA = X @ A
    YB = Y @ B    
    grad_A = -X.T @ YB / (n - 1)
    grad_B = -Y.T @ XA / (n - 1)
    return grad_A, grad_B

def cholesky_qr_retraction(X: np.ndarray, G: np.ndarray, xi: np.ndarray, epsilon=1e-6) -> np.ndarray:
    """ 
    Retract a point on the Stiefel manifold using the Cholesky QR retraction. Add regularization to avoid singular matrices.
    """
    Z = (X + xi).T @ G @ (X + xi)
    Z += epsilon * np.eye(Z.shape[0])  # Add regularization
    try:
        L = cholesky(Z, lower=True)
    except np.linalg.LinAlgError:
        # If Cholesky fails, fall back to Polar decomposition
        return polar_retraction(X, G, xi)
    retracted_point = (X + xi) @ np.linalg.inv(L.T)
    return retracted_point


def polar_retraction(X: np.ndarray, G: np.ndarray, xi: np.ndarray) -> np.ndarray:
    """ 
    Retract a point on the Stiefel manifold using the polar retraction.
    """
    Z = (X + xi).T @ G @ (X + xi)
    U, _ = polar(Z)
    retracted_point = (X + xi) @ np.linalg.inv(U.T)
    return retracted_point

In [64]:
import time 

def riemannian_gradient_descent(X, Y, k, retraction_method, max_iter=100, lr=0.1, tol=1e-6):
    n, p1 = X.shape
    _, p2 = Y.shape
    
    A = np.random.randn(p1, k)
    B = np.random.randn(p2, k)
    A, _ = np.linalg.qr(A)
    B, _ = np.linalg.qr(B)
    
    G_A = np.eye(p1)  # Metric for A
    G_B = np.eye(p2)  # Metric for B
    
    start_time = time.time()

    if np.isnan(A).any() or np.isnan(B).any() or np.isinf(A).any() or np.isinf(B).any():
        print("Error: Metric matrices A and B must be positive definite.")
        return None
    
    for i in range(max_iter):
        old_obj = cca_objective(X, Y, A, B)
        grad_A, grad_B = cca_gradient(X, Y, A, B)
        if np.isnan(grad_A).any() or np.isnan(grad_B).any():
            print("Error: Gradient matrices grad_A and grad_B must not have NaN values.")
            return None
        if np.isinf(grad_A).any() or np.isinf(grad_B).any():
            print("Error: Gradient matrices grad_A and grad_B must not have Inf values.")
            return None

        if retraction_method == 'cholesky':
            A = cholesky_qr_retraction(A, G_A, -lr * grad_A)
            B = cholesky_qr_retraction(B, G_B, -lr * grad_B)
        elif retraction_method == 'polar':
            A = polar_retraction(A, G_A, -lr * grad_A)
            B = polar_retraction(B, G_B, -lr * grad_B)
        
        new_obj = cca_objective(X, Y, A, B)
        if abs(new_obj - old_obj) < tol:
            break
    
    end_time = time.time()
    
    # Compute canonical correlations
    XA = X @ A
    YB = Y @ B
    correlations = np.diag(XA.T @ YB) / (n - 1)
    
    return A, B, correlations, end_time - start_time

def run_experiment(X, Y, k_values, retraction_methods):
    results = []
    for k in k_values:
        for method in retraction_methods:
            A, B, correlations, runtime = riemannian_gradient_descent(X, Y, k, method)
            results.append({
                'k': k,
                'method': method,
                'correlations': correlations,
                'runtime': runtime
            })
    return pd.DataFrame(results)

In [ ]:
# Load and preprocess data
scaler = StandardScaler()
X = scaler.fit_transform(X)
Y = scaler.fit_transform(Y)

# Run experiment
k_values = [5, 10, 15, 20]
retraction_methods = ['cholesky', 'polar']
results = run_experiment(X, Y, k_values, retraction_methods)

print(results)